In [ ]:
log_path = "backtests/2024-04-13_00-41-33.log"

In [45]:
import re
from io import StringIO

import pandas as pd
import plotly.express as px


def read_file_sections(filepath):
    section_delimiters = ["Sandbox logs:", "Activities log:", "Trade History:"]
    current_section = None
    data = {key: [] for key in section_delimiters}

    with open(filepath, "r") as file:
        for line in file:
            line = line.strip()
            if line in section_delimiters:
                current_section = line
                continue
            if current_section:
                data[current_section].append(line)

    return data


def extract_trades(text):
    # Regex to find all blocks enclosed in curly braces
    trade_pattern = r"\{[^{}]*\}"
    # Regex to extract key-value pairs within curly braces
    kv_pattern = r"\"(\w+)\":\s*(\"[^\"]*\"|\d+)"

    trades = []
    matches = re.findall(trade_pattern, text)
    for match in matches:
        # Extract key-value pairs from each match
        trade_data = {}
        for key, value in re.findall(kv_pattern, match):
            # Remove quotes from strings
            if value.startswith('"') and value.endswith('"'):
                value = value.strip('"')
            else:
                value = int(value)  # Convert numeric values to integers
            trade_data[key] = value
        trades.append(trade_data)
    return trades


sections = read_file_sections(log_path)
activities_data = "\n".join(sections["Activities log:"])
activities_df = pd.read_csv(StringIO(activities_data), delimiter=";")
trade_history_text = "\n".join(sections["Trade History:"])
trades = extract_trades(trade_history_text)
trade_history_df = pd.DataFrame(trades)

In [65]:
activities_df.isna().sum()

day                   0
timestamp             0
product               0
bid_price_1           0
bid_volume_1          0
bid_price_2        1782
bid_volume_2       1782
bid_price_3        3590
bid_volume_3       3590
ask_price_1           0
ask_volume_1          0
ask_price_2        1853
ask_volume_2       1853
ask_price_3        3594
ask_volume_3       3594
mid_price             0
profit_and_loss       0
dtype: int64

In [66]:
amethysts_df = activities_df[activities_df["product"] == "AMETHYSTS"]
starfruit_df = activities_df[activities_df["product"] == "STARFRUIT"]


# Price Trends Over Time for AMETHYSTS
fig_price_trends_amethysts = px.line(
    amethysts_df,
    x="timestamp",
    y="mid_price",
    title="Price Trends Over Time for AMETHYSTS",
    labels={"price": "Trade Price", "timestamp": "Timestamp"},
)
fig_price_trends_amethysts.show()

# Price Trends Over Time for STARFRUIT
fig_price_trends_starfruit = px.line(
    starfruit_df,
    x="timestamp",
    y="mid_price",
    title="Price Trends Over Time for STARFRUIT",
    labels={"price": "Trade Price", "timestamp": "Timestamp"},
)
fig_price_trends_starfruit.show()



# 1. Mid-Price Trends with Rolling Average
amethysts_df['mid_price_smooth'] = amethysts_df['mid_price'].rolling(window=3).mean()

fig_mid_price = px.line(
    amethysts_df,
    x='timestamp',
    y=['mid_price', 'mid_price_smooth'],
    title='Mid Price Trends for AMETHYSTS with Rolling Average',
    labels={'value': 'Mid Price', 'variable': 'Series'}
)
fig_mid_price.show()

# 2. Bid-Ask Spread Over Time
amethysts_df['spread'] = amethysts_df['ask_price_1'] - amethysts_df['bid_price_1']
fig_spread = px.line(
    amethysts_df,
    x='timestamp',
    y='spread',
    title='Bid-Ask Spread Over Time for AMETHYSTS',
    labels={'spread': 'Bid-Ask Spread'}
)
fig_spread.show()

# 3. Volume Trends Over Time
fig_volume = px.line(
    amethysts_df,
    x='timestamp',
    y=['bid_volume_1', 'ask_volume_1'],
    title='Bid and Ask Volume Trends for AMETHYSTS',
    labels={'value': 'Volume', 'variable': 'Volume Type'}
)
fig_volume.show()

# 4. Profit and Loss Over Time
fig_pnl = px.line(
    amethysts_df,
    x='timestamp',
    y='profit_and_loss',
    title='Profit and Loss Over Time for AMETHYSTS',
    labels={'profit_and_loss': 'Profit and Loss'}
)
fig_pnl.show()

# 5. Correlation Between Mid-Price Movement and Volume
fig_corr = px.scatter(
    amethysts_df,
    x='mid_price',
    y='bid_volume_1',
    title='Correlation Between Mid-Price and Bid Volume for AMETHYSTS',
    labels={'mid_price': 'Mid Price', 'bid_volume_1': 'Bid Volume'}
)
fig_corr.show()

/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_91402/3206840956.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_91402/3206840956.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



---

# Trade-History Ledger - Visualization

In [55]:
# Filter data for different products
amethysts_df = trade_history_df[trade_history_df["symbol"] == "AMETHYSTS"]
starfruit_df = trade_history_df[trade_history_df["symbol"] == "STARFRUIT"]

# Price Trends Over Time for AMETHYSTS
fig_price_trends_amethysts = px.line(
    amethysts_df,
    x="timestamp",
    y="price",
    title="Price Trends Over Time for AMETHYSTS",
    labels={"price": "Trade Price", "timestamp": "Timestamp"},
)
fig_price_trends_amethysts.show()

# Price Trends Over Time for STARFRUIT
fig_price_trends_starfruit = px.line(
    starfruit_df,
    x="timestamp",
    y="price",
    title="Price Trends Over Time for STARFRUIT",
    labels={"price": "Trade Price", "timestamp": "Timestamp"},
)
fig_price_trends_starfruit.show()

# Volume of Trades Over Time for AMETHYSTS
fig_volume_trades_amethysts = px.line(
    amethysts_df,
    x="timestamp",
    y="quantity",
    title="Volume of Trades Over Time for AMETHYSTS",
    labels={"quantity": "Trade Quantity", "timestamp": "Timestamp"},
)
fig_volume_trades_amethysts.show()

# Volume of Trades Over Time for STARFRUIT
fig_volume_trades_starfruit = px.line(
    starfruit_df,
    x="timestamp",
    y="quantity",
    title="Volume of Trades Over Time for STARFRUIT",
    labels={"quantity": "Trade Quantity", "timestamp": "Timestamp"},
)
fig_volume_trades_starfruit.show()